In [1]:
import time
import pandas as pd
import re
from selenium import webdriver
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

In [2]:
start_time = time.time()
print(f"start time: {time.strftime('%x %X')}")

start time: 02/28/20 09:54:54


In [3]:
def chrome():
    """This creates a chrome instance
    """
    option = webdriver.ChromeOptions()
    option.add_argument("--ignore-certificate-errors")
    option.add_argument("--ignore-ssl-errors")
    option.add_argument("--safebrowsing-disable-download-protection")
    option.add_experimental_option("excludeSwitches", ["enable-logging"])
    return webdriver.Chrome(options=option)


def midway(bot):
    """Takes chrome instance as input and returns midway authenticated chrome instance
    """
    print(
        "*" * 3,
        "Please Authenticate browser with your Midway credentials ",
        "*" * 3,
        "\n",
    )
    bot.get("https://midway.amazon.com")
    input("Please press enter once Midway Authentication is Successful: ")
    print(end="\n \n")
    return bot

In [4]:
asins = pd.read_excel(r"asins.xlsx").iloc[:, 0]

In [5]:
bot = midway(chrome())

*** Please Authenticate browser with your Midway credentials  *** 

Please press enter once Midway Authentication is Successful: 

 


In [17]:
bot.get("https://pricingrules.amazon.com/rest/itemDetails/1/1/null/B002Q9MCG8/latestPricingRulesEvaluationDetails")
page = bot.page_source
c.findall(page)

['Recommended Discount (Tax Included): 1.78 with discount strategy: COUPONS']

In [ ]:
c = re.compile(
    r"Recommended Discount \(Tax Included\): [.0-9]+ with discount strategy: [_A-Z]+")


def pricingDiscount(asin):
    try:
        bot.get(
            "https://pricingrules.amazon.com/rest/itemDetails/1/1/null/" + asin + "/latestPricingRulesEvaluationDetails")
        page = bot.page_source
        if len(c.findall(page)) != 0:
            return c.findall(page)
        else:
            return "Not Found"
    except NoSuchElementException:
        return "Not Found"
    except TimeoutException:
        return "Page Load Error"

In [ ]:
result = []
for i in asins:
    result.append([i, pricingDiscount(i)])
result = pd.DataFrame(result)
result.columns = ["asin", "pricingDiscount_comment"]
result.to_csv("pricing_remarks.csv", index=False)

In [ ]:
end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f"end time: {time.strftime('%x %X')}")
print(
    f"Approx. time taken : {round(minutes)} minutes {round(seconds)} seconds")